In [88]:
import pandas as pd
from sklearn.cluster import KMeans
import collections
import matplotlib.pyplot as plt
from matplotlib import colors
import pylab as pl
%matplotlib inline

import scipy.stats as stats

import sklearn
import statsmodels.api as sm

import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

#special matplotlib argument for improved plots
from matplotlib import rcParams

In [89]:
data = pd.read_csv(r'C:\Users\PB\Downloads\CarsNew.csv')
data = data.loc[data['Year'].isin([2012, 2013, 2014, 2015, 2016, 2017, 2018])]
data = data.loc[data['Transmission Type'].isin(['MANUAL','AUTOMATIC'])]

In [90]:
logdata = data.fillna(0)
logdata.head(1)

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
5,BMW,1 Series,2012,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,31200


In [91]:
logdata.shape

(6697, 16)

In [92]:
logdata.keys()

Index(['Make', 'Model', 'Year', 'Engine Fuel Type', 'Engine HP',
       'Engine Cylinders', 'Transmission Type', 'Driven_Wheels',
       'Number of Doors', 'Market Category', 'Vehicle Size', 'Vehicle Style',
       'highway MPG', 'city mpg', 'Popularity', 'MSRP'],
      dtype='object')

In [93]:
columns = ['Model','Year', 'Engine Fuel Type', 'Driven_Wheels', 'Vehicle Style', 'Popularity', 'MSRP']

#remove white spaces in columns names:
logdata.columns = logdata.columns.str.strip()

#drop unwated columns:
logdata.drop(columns, inplace=True, axis=1)

logdata.columns = ['make', 'hp', 'cylinders', 'transmission', 'doors','Market Category' , 'carsize', 'highway_mpg',  'city_mpg']

logdata.head()


,make,hp,cylinders,transmission,doors,Market Category,carsize,highway_mpg,city_mpg
5,BMW,230.0,6.0,MANUAL,2.0,"Luxury,Performance",Compact,28,18
6,BMW,300.0,6.0,MANUAL,2.0,"Luxury,Performance",Compact,26,17
7,BMW,300.0,6.0,MANUAL,2.0,"Luxury,High-Performance",Compact,28,20
8,BMW,230.0,6.0,MANUAL,2.0,Luxury,Compact,28,18
9,BMW,230.0,6.0,MANUAL,2.0,Luxury,Compact,27,18


In [94]:
logdata.transmission = pd.Categorical(logdata.transmission)
logdata['transmission_code'] = logdata.transmission.cat.codes
logdata.head(5)

,make,hp,cylinders,transmission,doors,Market Category,carsize,highway_mpg,city_mpg,transmission_code
5,BMW,230.0,6.0,MANUAL,2.0,"Luxury,Performance",Compact,28,18,1
6,BMW,300.0,6.0,MANUAL,2.0,"Luxury,Performance",Compact,26,17,1
7,BMW,300.0,6.0,MANUAL,2.0,"Luxury,High-Performance",Compact,28,20,1
8,BMW,230.0,6.0,MANUAL,2.0,Luxury,Compact,28,18,1
9,BMW,230.0,6.0,MANUAL,2.0,Luxury,Compact,27,18,1


In [96]:
logdata = logdata.fillna('other')
logdata.head(5)

ValueError: fill value must be in categories

In [79]:
logdata.loc[logdata['Market Category'].str.contains('Luxury')]

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
5,BMW,1 Series,2012,premium unleaded (required),230,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,31200
6,BMW,1 Series,2012,premium unleaded (required),300,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,26,17,3916,44100
7,BMW,1 Series,2012,premium unleaded (required),300,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,39300
8,BMW,1 Series,2012,premium unleaded (required),230,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,36900
9,BMW,1 Series,2013,premium unleaded (required),230,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,27,18,3916,37200
10,BMW,1 Series,2013,premium unleaded (required),300,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,39600
11,BMW,1 Series,2013,premium unleaded (required),230,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,19,3916,31500
12,BMW,1 Series,2013,premium unleaded (required),300,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,44400
13,BMW,1 Series,2013,premium unleaded (required),230,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,19,3916,37200
14,BMW,1 Series,2013,premium unleaded (required),230,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,19,3916,31500


In [67]:
logdata.rename(columns={'Vehicle Size': 'carsize'}, inplace=True)
logdata.rename(columns={'highway MPG': 'highmpg'}, inplace=True)
logdata.rename(columns={'city mpg': 'citympg'}, inplace=True)
logdata.rename(columns={'Engine Cylinders': 'cylinders'}, inplace=True)
logdata.rename(columns={'Transmission Type': 'transmission'}, inplace=True)
logdata.rename(columns={'Engine Fuel Type': 'fuel'}, inplace=True)
logdata.rename(columns={'Engine HP': 'hp'}, inplace=True)
logdata.rename(columns={'Number of Doors': 'doors'}, inplace=True)
logdata.rename(columns={'Vehicle Style': 'style'}, inplace=True)
logdata.rename(columns={'Market Category': 'category'}, inplace=True)
logdata.head(5)

,Make,Model,Year,fuel,hp,cylinders,transmission,Driven_Wheels,doors,category,carsize,style,highmpg,citympg,Popularity,MSRP
5,BMW,1 Series,2012,premium unleaded (required),230,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,31200
6,BMW,1 Series,2012,premium unleaded (required),300,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,26,17,3916,44100
7,BMW,1 Series,2012,premium unleaded (required),300,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,39300
8,BMW,1 Series,2012,premium unleaded (required),230,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,36900
9,BMW,1 Series,2013,premium unleaded (required),230,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,27,18,3916,37200


In [69]:
logdata.carsize = pd.Categorical(logdata.carsize)
logdata['size_code'] = logdata.carsize.cat.codes
logdata.head(5)

,Make,Model,Year,fuel,hp,cylinders,transmission,Driven_Wheels,doors,category,carsize,style,highmpg,citympg,Popularity,MSRP,size_code
5,BMW,1 Series,2012,premium unleaded (required),230,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,31200,0
6,BMW,1 Series,2012,premium unleaded (required),300,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,26,17,3916,44100,0
7,BMW,1 Series,2012,premium unleaded (required),300,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,39300,0
8,BMW,1 Series,2012,premium unleaded (required),230,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,36900,0
9,BMW,1 Series,2013,premium unleaded (required),230,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,27,18,3916,37200,0


In [70]:
logdata.make = pd.Categorical(logdata.make)
logdata['make_code'] = logdata.make.cat.codes
logdata.head(5)

AttributeError: 'DataFrame' object has no attribute 'make'

In [58]:
col = ['carsize','transmission','make']
logdata.drop(col, inplace=True, axis=1)
logdata.head()

KeyError: "labels ['carsize' 'transmission' 'make'] not contained in axis"

In [12]:
logdata.shape

(6697, 8)

In [13]:
logdata.describe()

,hp,cylinders,doors,highway_mpg,city_mpg,transmission_code,size_code,make_code
count,6697.000000,6697.000000,6697.000000,6697.000000,6697.000000,6697.000000,6697.000000,6697.000000
mean,270.717187,5.559803,3.604151,27.849783,20.440944,0.174705,1.069583,18.436165
std,108.425612,1.730684,0.782462,6.275163,5.972094,0.379743,0.858715,11.652532
min,0.000000,3.000000,2.000000,13.000000,10.000000,0.000000,0.000000,0.000000
25%,184.000000,4.000000,4.000000,23.000000,17.000000,0.000000,0.000000,7.000000
50%,268.000000,6.000000,4.000000,27.000000,19.000000,0.000000,1.000000,15.000000
75%,325.000000,6.000000,4.000000,32.000000,23.000000,0.000000,2.000000,28.000000
max,707.000000,12.000000,4.000000,53.000000,58.000000,1.000000,2.000000,37.000000


In [14]:
logdata.keys()
#kmeans_data.brand.cat.categories

Index(['hp', 'cylinders', 'doors', 'highway_mpg', 'city_mpg',
       'transmission_code', 'size_code', 'make_code'],
      dtype='object')

In [16]:
logdata.head(1)

,hp,cylinders,doors,highway_mpg,city_mpg,transmission_code,size_code,make_code
5,230.0,6.0,2.0,28,18,1,0,3


In [22]:
import math;
logdata['Catalogue']= logdata['hp'] + logdata['size_code'] /2
logdata.head(50)

,hp,cylinders,doors,highway_mpg,city_mpg,transmission_code,size_code,make_code,Catalogue
5,230.0,6.0,2.0,28,18,1,0,3,230.0
6,300.0,6.0,2.0,26,17,1,0,3,300.0
7,300.0,6.0,2.0,28,20,1,0,3,300.0
8,230.0,6.0,2.0,28,18,1,0,3,230.0
9,230.0,6.0,2.0,27,18,1,0,3,230.0
10,300.0,6.0,2.0,28,20,1,0,3,300.0
11,230.0,6.0,2.0,28,19,1,0,3,230.0
12,300.0,6.0,2.0,28,19,1,0,3,300.0
13,230.0,6.0,2.0,28,19,1,0,3,230.0
14,230.0,6.0,2.0,28,19,1,0,3,230.0
